# Prepare Historical

Uses:

- resources/historical.xlsx

Generates:

- resources/historical_join.xlsx



In [1]:
import sys  
sys.path.insert(1, '..')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import os
from util import HISTORICAL_FILE, HISTORICAL_DB_KEEP_OUT, RESOURCE_DIR, IMAGES_DIR

In [3]:
#reads projects from Excel file
df = pd.read_excel(HISTORICAL_FILE, keep_default_na=False)
df.head()

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,ClickHouse,Kdb+,Virtuoso-NoSql,HBase,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,Enterprise Resource Planning,38b284602be9dcb6b4253cfa1c2f9420c002b56a,2,2010-08-03 08:31:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,Enterprise Resource Planning,fbcdc18a66e7f7960a7ed966f87b1527c5c7551a,3,2010-09-02 09:45:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,Enterprise Resource Planning,8d05dda18ce4c5db6f30cf01d7e0c2059b1a5388,4,2010-09-16 15:33:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,Enterprise Resource Planning,a789ba2b30383777b4d0d81bbc62a90a24dbb032,5,2010-09-23 13:38:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.columns

Index(['owner', 'name', 'domain', 'sha1', 'part_commit', 'date_commit',
       'isLast', 'SapHana', 'Realm', 'SAP SQL Anywhere', 'FileMaker',
       'Virtuoso', 'ArangoDB', 'Informix', 'HyperSQL', 'CouchDB', 'Riak KV',
       'Ignite', 'Virtuoso-Sql', 'Neo4j', 'Singlestore', 'Snowflake',
       'Ignite-Sql', 'MarkLogic', 'Hazelcast', 'GoogleCloudFirestore',
       'Influx DB', 'H2', 'Ignite-NoSql', 'Interbase', 'Teradata', 'Oracle',
       'Impala', 'Microsoft Azure CosmosDB', 'MySQL_Maria DB', 'MS Access',
       'Microsoft Azure Table Storage', 'Aerospike', 'IBM DB2', 'DynamoDB',
       'Vertica', 'Ingres', 'MongoDB',
       'MS SQL Server_Microsoft Azure SQL Database', 'Google Cloud Datastore',
       'Firebase Realtime', 'SQLite', 'PostgreSQL_ CockroachDB',
       'SAP Adaptive Server', 'ClickHouse', 'Kdb+', 'Virtuoso-NoSql', 'HBase',
       'Firebird', 'Cassandra', 'Couchbase', 'Etcd', 'PostGIS', 'Redis'],
      dtype='object')

In [5]:
df["Join_Ignite-Sql"] = df["Ignite"] +  df["Ignite-Sql"]
df["Join_Ignite-NoSql"] = df["Ignite"] +  df["Ignite-NoSql"]
#df["Join_MarkLogic-Sql"] = df["MarkLogic"] +  df["MarkLogic-Sql"]
#df["Join_MarkLogic-NoSql"] = df["MarkLogic"] +  df["MarkLogic-NoSql"]
df["Join_Virtuoso-Sql"] = df["Virtuoso"] +  df["Virtuoso-Sql"] #(Não virtuoso SQL como resultado)
df["Join_Virtuoso-NoSql"] = df["Virtuoso"] +  df["Virtuoso-NoSql"]

df.rename(columns={'MySQL_Maria DB': 'MySQL'}, inplace=True)
df.rename(columns={'PostgreSQL_ CockroachDB': 'PostgreSQL'}, inplace=True)
df.rename(columns={'MS SQL Server_Microsoft Azure SQL Database': 'MS SQL Server'}, inplace=True)
df.rename(columns={'GoogleCloudFilestore': 'GoogleCloudFirestore'}, inplace=True)
#df.rename(columns={'Join_Ignite-Sql': 'Ignite-Sql'}, inplace=True)
#df.rename(columns={'Join_Ignite-NoSql': 'Ignite-NoSql'}, inplace=True)


df

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,Enterprise Resource Planning,38b284602be9dcb6b4253cfa1c2f9420c002b56a,2,2010-08-03 08:31:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,Enterprise Resource Planning,fbcdc18a66e7f7960a7ed966f87b1527c5c7551a,3,2010-09-02 09:45:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,Enterprise Resource Planning,8d05dda18ce4c5db6f30cf01d7e0c2059b1a5388,4,2010-09-16 15:33:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,Enterprise Resource Planning,a789ba2b30383777b4d0d81bbc62a90a24dbb032,5,2010-09-23 13:38:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31111,liferay,liferay-portal,Enterprise Resource Planning,fcd4cd6526c80613eca69927effe931bb6a91444,8209,2024-10-07 12:05:23+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31112,liferay,liferay-portal,Enterprise Resource Planning,cc0ab8103195e543056639c8c2883fcc187c33a2,8210,2024-10-07 17:58:58+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31113,liferay,liferay-portal,Enterprise Resource Planning,f75082029372ab28694650bd3fe9fae4d548351e,8211,2024-10-11 04:36:52+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31114,liferay,liferay-portal,Enterprise Resource Planning,a80a679a86d2867f985bafa510d9d12f1bf051a7,8212,2024-10-11 16:53:22+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [6]:
df.drop(["Ignite", "Ignite-Sql","Ignite-NoSql", #"MarkLogic","MarkLogic-Sql","MarkLogic-NoSql", 
          "Virtuoso", "Virtuoso-Sql", "Virtuoso-NoSql"], axis=1, inplace=True)

In [7]:
df['Join_Ignite-Sql']= df['Join_Ignite-Sql'].replace([2,1],[1,0])
df['Join_Ignite-NoSql']= df['Join_Ignite-NoSql'].replace([2,1],[1,0])
#df['Join_MarkLogic-Sql']= df['Join_MarkLogic-Sql'].replace([2,1],[1,0])
#df['Join_MarkLogic-NoSql']= df['Join_MarkLogic-NoSql'].replace([2,1],[1,0])
df['Join_Virtuoso-Sql']= df['Join_Virtuoso-Sql'].replace([2,1],[1,0])
df['Join_Virtuoso-NoSql']= df['Join_Virtuoso-Sql'].replace([2,1],[1,0])

In [8]:
HISTORICAL_FILE_JOIN = RESOURCE_DIR + os.sep + 'historical_join.xlsx'
df.to_excel(HISTORICAL_FILE_JOIN, index=False)